<b>PySpark RandomForestClassifier Fraud Detection using a highly unbalanced dataset from Kaggle</b><br>
No feature engineering allowing for a concise notebook illustrating Hyperparameter Tuning & Training with AWS EMR

https://www.kaggle.com/mlg-ulb/creditcardfraud

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.classification import RandomForestClassifier

estimator = RandomForestClassifier()
paramGrid = ParamGridBuilder().addGrid(estimator.numTrees, [5, 10]).build()

raw_data              = SparkSession.builder.getOrCreate().read.csv('s3://creditcardfraud123/creditcard.csv', header=True, inferSchema=True)
assembler             = VectorAssembler(inputCols=['Time','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount'], outputCol='features')
train_data, test_data = assembler.transform(raw_data.withColumnRenamed('Class', 'label')).select('features', 'label').randomSplit([0.7,0.3])
lr_predictions        = CrossValidator(estimator          = estimator                      ,
                                       estimatorParamMaps = paramGrid                      ,
                                       evaluator          = BinaryClassificationEvaluator(),
                                       numFolds           = 2
                                      ).fit(train_data).transform(test_data)
metrics               = MulticlassMetrics(lr_predictions.select(lr_predictions['prediction'], lr_predictions['label'].cast('double')).rdd)

print('Accuracy  = %s' % round(metrics.accuracy   , 2))
print('Precision = %s' % round(metrics.precision(), 2))
print('Recall    = %s' % round(metrics.recall()   , 2))
print('F1 Score  = %s' % round(metrics.fMeasure() , 2))

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1573519027292_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy  = 1.0
Precision = 1.0
Recall    = 1.0
F1 Score  = 1.0